指定输入视频路径，对视频中每一帧进行OCR和马赛克处理，输出一段打了马赛克的新视频。

同济子豪兄 2022-1-6

# 切换工作目录到主目录

In [1]:
import os
os.chdir('../')
os.listdir()

['.git',
 '.circleci',
 '.codespellrc',
 '.coveragerc',
 '.github',
 '.gitignore',
 '.pre-commit-config.yaml',
 '.pylintrc',
 '.readthedocs.yml',
 'CITATION.cff',
 'LICENSE',
 'MANIFEST.in',
 'README.md',
 'README_zh-CN.md',
 'configs',
 'demo',
 'docker',
 'docs',
 'mmocr',
 'model-index.yml',
 'requirements.txt',
 'requirements',
 'resources',
 'setup.cfg',
 'setup.py',
 'tests',
 'tools',
 '__MACOSX',
 '.eggs',
 'outputs',
 'tutorials',
 '.ipynb_checkpoints',
 'dict_printed_chinese_english_digits.txt',
 'font.TTF',
 'dict_printed_chinese_english_digits.txt.1']

# 处理单帧的函数

In [2]:
# 导入pytorch
import torch, torchvision

# 导入mmcv和mmdet
import mmcv
import mmdet

import numpy as np

# 导入opencv-python
import cv2

from mmocr.utils.ocr import MMOCR

mmocr = MMOCR(det='TextSnake', recog=None,device='cpu')

load checkpoint from http path: https://download.openmmlab.com/mmocr/textdet/textsnake/textsnake_r50_fpn_unet_1200e_ctw1500-27f65b64.pth


In [3]:
def mosaic_img(img):
    result = mmocr.readtext(img, output=None)

    for each in result[0]['boundary_result']: # 遍历每个文字区域
        if each[-1] > 0.2: # 置信度阈值
            point_coords = np.array(each[:-1]).reshape(-1,2)
            point_coords = np.array([point_coords], dtype = np.int32)
            cv2.fillPoly(img, point_coords, (0,255,0)) # 多边形填充
        else:
            print('有一片文字区域置信度为{:.2f}，小于阈值，不打码'.format(each[-1]))
    return img

In [4]:
# 处理帧函数
def process_frame(img):
    
    # 记录该帧开始处理的时间
    start_time = time.time()
    scaler = 1 # 文字大小
    
    img = mosaic_img(img)
    
    
    # 记录该帧处理完毕的时间
    end_time = time.time()
    # 计算每秒处理图像帧数FPS
    FPS = 1/(end_time - start_time)  

    img = cv2.putText(img, 'FPS  '+str(int(FPS)), (25 * scaler, 100 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)


    return img

In [5]:
# 调用摄像头逐帧实时处理模板
# 不需修改任何代码，只需修改process_frame函数即可
# 同济子豪兄 2021-7-8

# 导入opencv-python
import cv2
import time

# 获取摄像头，传入0表示获取系统默认摄像头
cap = cv2.VideoCapture(1)

# 打开cap
cap.open(0)

# 无限循环，直到break被触发
while cap.isOpened():
    # 获取画面
    success, frame = cap.read()
    if not success:
        print('Error')
        break
    
    ## !!!处理帧函数
    frame = process_frame(frame)
    
    # 展示处理后的三通道图像
    cv2.imshow('my_window',frame)

    if cv2.waitKey(1) in [ord('q'),27]: # 按键盘上的q或esc退出（在英文输入法下）
        break
    
# 关闭摄像头
cap.release()

# 关闭图像窗口
cv2.destroyAllWindows()

[ WARN:0] global /tmp/pip-req-build-3129w7z7/opencv/modules/videoio/src/cap_v4l.cpp (890) open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ WARN:0] global /tmp/pip-req-build-3129w7z7/opencv/modules/videoio/src/cap_v4l.cpp (890) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


# 视频逐帧处理（模板）¶

In [6]:
# 视频逐帧处理代码模板
# 不需修改任何代码，只需定义process_frame函数即可
# 同济子豪兄 2021-7-10
from tqdm import tqdm
import time

def generate_video(input_path='./videos/three-hands.mp4'):
    filehead = input_path.split('/')[-1]
    output_path = "out-" + filehead
    
    print('视频开始处理',input_path)
    
    # 获取视频总帧数
    cap = cv2.VideoCapture(input_path)
    frame_count = 0
    while(cap.isOpened()):
        success, frame = cap.read()
        frame_count += 1
        if not success:
            break
    cap.release()
    print('视频总帧数为',frame_count)
    
    # cv2.namedWindow('Crack Detection and Measurement Video Processing')
    cap = cv2.VideoCapture(input_path)
    frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
    # fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)

    out = cv2.VideoWriter(output_path, fourcc, fps, (int(frame_size[0]), int(frame_size[1])))
    
    # 进度条绑定视频总帧数
    with tqdm(total=frame_count-1) as pbar:
        try:
            while(cap.isOpened()):
                success, frame = cap.read()
                if not success:
                    break

                # 处理帧
                # frame_path = './temp_frame.png'
                # cv2.imwrite(frame_path, frame)
                try:
                    frame = process_frame(frame)
                except Exception as e:
                    print(e)
                    pass
                
                if success == True:
                    # cv2.imshow('Video Processing', frame)
                    out.write(frame)

                    # 进度条更新一帧
                    pbar.update(1)

                # if cv2.waitKey(1) & 0xFF == ord('q'):
                    # break
        except:
            print('中途中断')
            pass

    cv2.destroyAllWindows()
    out.release()
    cap.release()
    print('视频已保存', output_path)

In [7]:
generate_video(input_path='tutorials/videos/test2.mp4')

视频开始处理 tutorials/videos/test2.mp4
视频总帧数为 78


  0%|          | 0/77 [00:00<?, ?it/s]/environment/miniconda3/lib/python3.7/site-packages/mmdet/datasets/utils.py:69: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)
100%|██████████| 77/77 [04:39<00:00,  3.63s/it]

视频已保存 out-test2.mp4
